<a href="https://colab.research.google.com/github/LeeIsBadK/CovidManchaSEIR/blob/main/CovidManchaSEIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# โมเดลแนวโน้มจำนวนผู้ติดเชื้อ COVID-19 ภายในอำเภอมัญจาคีรี โดยใช้โมเดล SEIR
---
กิตติพัฒน์ ลี้สมบัติวัฒนะ

โรงเรียนมัญจาศึกษา อำเภอมัญจาคีรี จังหวัดขอนแก่น

## คำชี้แจง
เมื่อต้องการจำลองโมเดล ให้กด Runtime -> Run all หรือ กดทำงานในช่อง coding ทีละบรรทัด และควรกดทุกครั้งที่มีการเปลี่ยนค่าพารามิเตอร์ 

In [ ]:
# Submitted By Kittiphat Leesombatwathana
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
print('Import library done')

## สมมุติฐานเบื้องต้น
1. ประชากรในอำเภอมัญจาคีรีไม่มีการเคลื่อนย้ายประชากร รวมถึงไม่มีการเกิด และ การตาย
2. ทุกคนมีความเสี่ยงต่อการติดเชื้อเท่าๆ กัน อายุและเพศ จะไม่มีผล
3. มีจำนวนผู้ที่สามารถแพร่เชื้อเริ่มต้นเพียงคนเดียว


## พารามิเตอร์
ค่าพารามิเตอร์จะอ้างอิงกับงานวิจัยนี้ (https://www.nejm.org/doi/full/10.1056/NEJMoa2001316)
1. ระยะเวลาในการฟักตัวของเชื้อเฉลี่ย  = 3.2 วัน
2. ระยะเวลาการติดเชื้อเฉลี่ย  = 2.3 วัน 
3. ระดับค่าการติดเชื้อ (R0) = 2.006
4. ประชากรในอำเภอมัญจาคีรี จังหวัดขอนแก่นในพ.ศ. 2563-2564 = 71371 คน (สำนักงานสถิติแห่งชาติ)


In [ ]:
#Parameter
t_incubation = 5.2
t_infective = 3.2
R0 = 2.2
N = 71371
Display = 200

print('Done')

## การออกมาตรการควบคุมโรค
เราจะใช้ค่า u เป็นตัวกำหนดร้อยละที่ลดจำนวนผู้เสี่ยงติดเชื้อได้ โดยมีค่าตั้งแต่ 0-1

In [ ]:
# u is social distancing indicator
# u = 0   no social distancing
# u = 0.1 ware mask
# u = 0.2 ware mask and often cleaning
# u = 0.3 ware mask, often cleaning and WFH
u = 0

print('Done')

## ส่วนการคำนวณ และ Coding

In [ ]:
# Define var
e0= 1/N
i0 = 0.0
r0 = 0.0
s0 = 1 - i0 - r0 - e0
x0 = [s0, e0, i0, r0]

alpha = 1/t_incubation
gramma = 1/t_infective
beta = R0 * gramma
hm = 1 - 1/R0

In [ ]:
# Define differential equation
def covid(x,t):
    s, e, i, r = x
    dx = np.zeros(4)
    dx[0] = -(1-u) * beta * s * i
    dx[1] = (1-u) * beta * s * i - (alpha * e)
    dx[2] = alpha * e - ( gramma * i)
    dx[3] = gramma * i
    return dx

In [ ]:
# Run differential equation with var
t = np.linspace(0,Display,Display)
x = odeint(covid,x0,t)
s = x[:,0]; e = x[:,1]; i = x[:,2]; r = x[:,3]

In [ ]:
# plot the graph fron differential equation
plt.figure(figsize=(8,5))

plt.subplot(2,1,1)
plt.title('Social distancing = '+str(u*100)+'%')
plt.plot(t,s*N,color='blue', lw=3, label='Susceptible')
plt.plot(t,r*N,color='green', lw=3, label='Recovered')
plt.axhline(hm*N,lw=1)
plt.ylabel('People')
plt.legend()

plt.subplot(2,1,2)
plt.plot(t,i*N,color='red', lw=3, label='Infective')
plt.plot(t,e*N,color='orange', lw=3, label='Exposed')
plt.ylim(0,N/5)
plt.xlabel('Time (Per days)')
plt.ylabel('People')
plt.legend()

plt.show()

In [ ]:
# Display dataframe with differential equation
dt = x * N
df = pd.DataFrame(data=dt, columns=['Susceptible','Exposed','Infective','Recovered'],index = [str(i+1) for i in range(Display)]).round().astype(int)
df

## Herd immunity ในพื้นที่อำเภอมัญจาคีรี
การเกิดภูมิคุ้มกันหมู่จะเกิดก็ต่อเมื่อมีจำนวนผู้ติดเชื้อในร้อยละ $$ 1-\frac{1}{R_0}$$
ซึ่งจะเกิดในวันที่ผู้ติดเชื้อมากที่สุด ก่อนที่จะมีผู้ติดเชื้อลดลง

In [ ]:
# maximun infective in 1 day
print('Maximum Infectives in day : ',int(df["Infective"].idxmax()),
      '\n Infective : ',df["Infective"].max(),
      '\n Recovered : ',df.loc[df["Infective"].idxmax(), 'Recovered'])

In [ ]:
# The sum of maximun infective and recovered in the maximum infective day
print(df["Infective"].max() + df.loc[df["Infective"].idxmax(), 'Recovered'])

In [ ]:
# Real herd immunity equation
print((1-(1/R0))*N)

## การบันทึกไฟล์เป็น CSV เพื่อนำไป Visualization


In [ ]:
# df.to_csv(r'#local file where you want to save#\#name#.csv', index = False) then delete the '#' in front of code
# df.to_csv(r'C:\Users\User\Documents\Covid data visual\ManchaKhiriModel.csv', index = False)